In [1]:
from fastai.vision import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callbacks import SaveModelCallback

if torch.cuda.is_available():
  print("Great! Good to go!")
else:
  print('CUDA is not up!')

Great! Good to go!


In [2]:
from gepcore.utils import cell_graph, convolution
from gepcore.entity import Gene, Chromosome
from gepcore.symbol import PrimitiveSet
from gepnet.model import get_gepnet, arch_config
from gepnet.utils import count_parameters
torch.backends.cudnn.benchmark = True

In [3]:
from pygraphviz import AGraph
import glob

In [ ]:
# get chromosme from fil1
gpath = '/home/cliff/ResearchProjects/geppy_nn/mlj_experiments/3-2-train/3-2-seed-2/best/indv_10'
graph = [AGraph(g) for g in glob.glob(gpath+'/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)

#cell_graph.draw_graph(graph, 'nb_graphs/rs/run_4')
print(comp_graph)

In [ ]:
# # generate new chromosome
# # define primitive set
# pset = PrimitiveSet('cnn')

# # add cellular encoding program symbols
# pset.add_program_symbol(cell_graph.end)
# pset.add_program_symbol(cell_graph.seq)
# pset.add_program_symbol(cell_graph.cpo)
# pset.add_program_symbol(cell_graph.cpi)

# # add convolutional operations symbols
# conv_symbol = convolution.get_symbol()
# pset.add_cell_symbol(conv_symbol.conv1x1)
# pset.add_cell_symbol(conv_symbol.conv3x3)
# pset.add_cell_symbol(conv_symbol.dwconv3x3)
# #pset.add_cell_symbol(conv_symbol.conv1x3)
# #pset.add_cell_symbol(conv_symbol.conv3x1)
# #pset.add_cell_symbol(conv_symbol.maxpool3x3)

# def gene_gen():
#     return Gene(pset, 3)

# ch = Chromosome(gene_gen, 4)
# graph, comp_graph = cell_graph.generate_comp_graph(ch)

# cell_graph.save_graph(graph, 'nb_graphs/rs/run_4')
# cell_graph.draw_graph(graph, 'nb_graphs/rs/run_4')

In [ ]:
seed = 331
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=16,
                   repeat_list=[3, 2, 1, 2],
                   classes=30)

net = get_gepnet(conf)
count_parameters(net)

In [ ]:
path = Path("/home/cliff/rs_imagery/AID")
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

bs = 128
data = (ImageList.from_folder(path/'train')
        .split_by_rand_pct(valid_pct=0.1, seed=seed) 
        .label_from_folder()
        .transform(tfms, size=224)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize())

In [ ]:
#model_dir = '/home/cliff/ResearchProjects/models/random_search/'
learn = Learner(data, net, metrics=[error_rate, accuracy], model_dir=gpath).mixup()
learn.to_fp16()

In [ ]:
learn.lr_find(end_lr=100)

learn.recorder.plot()

In [ ]:
cb = SaveModelCallback(learn, every='improvement', monitor='accuracy', name='aid-model-s322')
learn.fit_one_cycle(500, 1e-2, wd=0.0004, callbacks=[cb]) 

In [ ]:
################################# Testing ########################################

In [4]:
gpath = '/home/cliff/ResearchProjects/geppy_nn/mlj_experiments/3-2-train/3-2-seed-2/best/indv_10'
graph = [AGraph(g) for g in glob.glob(gpath+'/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)


conf = arch_config(comp_graph=comp_graph,
               depth_coeff=1.0,
               width_coeff=1.0,
               channels=16,
               repeat_list=[3, 2, 1, 2],
               classes=30)

net = get_gepnet(conf)

# classifier = nn.Sequential(nn.AdaptiveAvgPool2d(1), 
#                            Flatten(),
#                            init_default(nn.Linear(num_ftrs, num_ftrs*2), nn.init.kaiming_normal_),
#                            #nn.Dropout2d(0.5),
#                            #init_default(nn.Linear(num_ftrs*2, num_ftrs*2), nn.init.kaiming_normal_),
#                            #nn.Dropout2d(0.5),
#                            init_default(nn.Linear(num_ftrs*2, 30), nn.init.kaiming_normal_))
# net.head = classifier
print(count_parameters(net),'\n')


for i in range(10):

    tfms = get_transforms(do_flip=False)
    path = Path("/home/cliff/rs_imagery/AID/")

    bs = 128

    data = (ImageList.from_folder(path)
            .split_by_folder(train='train', valid='test')
            .label_from_folder()
            .transform(tfms, size=224)
            .databunch(bs=bs, num_workers=num_cpus())
            .normalize())

    model = Learner(data, net, metrics=[accuracy, error_rate]).load(gpath+'/aid-model-s322')
    _, acc, _ = model.validate()
    print('%.2f' %(acc.item()*100))

5.476558 



95.35


94.85


95.15


95.00


...

94.80


95.00


95.05


95.20


95.35


In [ ]:
# model.validate(data.valid_dl)

In [ ]:
# interp = ClassificationInterpretation.from_learner(model)
# losses,idxs = interp.top_losses()
# len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
#interp.plot_top_losses(9, figsize=(15,15), heatmap=False)

In [ ]:
# interp.plot_confusion_matrix(normalize=True, plot_txt=True, figsize=(20,16), 
#                              dpi=225, norm_dec=2)

In [ ]:
#preds,y,losses = model.get_preds(ds_type=DatasetType.Valid, with_loss=True)
#preds[:5]

In [ ]:
############################## Fine-tuning ##################################

In [4]:
gpath = '/home/cliff/ResearchProjects/geppy_nn/mlj_experiments/3-2-train/3-2-seed-2/best/indv_10'
graph = [AGraph(g) for g in glob.glob(gpath+'/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)


conf = arch_config(comp_graph=comp_graph,
               depth_coeff=1.0,
               width_coeff=1.0,
               channels=16,
               repeat_list=[3, 2, 1, 2],
               classes=45)

net = get_gepnet(conf)
print(count_parameters(net),'\n')

5.480413 



In [5]:
path = Path("/home/cliff/rs_imagery/AID-50-50")
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

bs = 128
data = (ImageList.from_folder(path/'train')
        .split_by_rand_pct(valid_pct=0.1, seed=322) 
        .label_from_folder()
        .transform(tfms, size=224)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize())

learn = Learner(data, net, metrics=[error_rate, accuracy], model_dir=gpath).load(gpath+'/best-model-322')

In [6]:
for param in learn.model.parameters():
    param.requires_grad = False

In [7]:
num_ftrs = learn.model.head[2].in_features
classifier = nn.Sequential(nn.AdaptiveAvgPool2d(1), 
                           Flatten(),
                           #init_default(nn.Linear(num_ftrs, num_ftrs*2), nn.init.kaiming_normal_),
                           #nn.Dropout2d(0.5),
                           #init_default(nn.Linear(num_ftrs*2, num_ftrs*2), nn.init.kaiming_normal_),
                           nn.Dropout2d(0.5),
                           init_default(nn.Linear(num_ftrs, 30), nn.init.kaiming_normal_))

learn.model.head = classifier

In [8]:
learn = Learner(data, net, metrics=[error_rate, accuracy], model_dir=gpath).mixup()
#learn.to_fp16()

In [9]:
learn.summary()
#learn.model.cuda()

GepNet
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [16, 224, 224]       432        False     
______________________________________________________________________
BatchNorm2d          [16, 224, 224]       32         False     
______________________________________________________________________
ReLU                 [16, 224, 224]       0          False     
______________________________________________________________________
AvgPool2d            [16, 112, 112]       0          False     
______________________________________________________________________
Conv2d               [32, 112, 112]       4,608      False     
______________________________________________________________________
BatchNorm2d          [32, 112, 112]       64         False     
______________________________________________________________________
ReLU                 [32, 112, 112]       0          False     
_______________________________________________________

In [9]:
cb = SaveModelCallback(learn, every='improvement', monitor='accuracy', name='aid-model-e322')
learn.fit_one_cycle(500, 1e-2, wd=0.0004, callbacks=[cb]) 
#learn.fit_fc(500, 1e-2, wd=0.0004, callbacks=[cb]) 

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,3.393881,3.356523,0.930000,0.070000,00:54
1,3.363288,3.305961,0.854000,0.146000,00:21
2,3.334419,3.260925,0.772000,0.228000,00:19
3,3.299351,3.213364,0.696000,0.304000,00:19
4,3.266691,3.168009,0.632000,0.368000,00:19
5,3.228932,3.122909,0.574000,0.426000,00:19
6,3.193622,3.076479,0.504000,0.496000,00:19
7,3.154604,3.027080,0.446000,0.554000,00:19
8,3.119541,2.982888,0.400000,0.600000,00:19
9,3.081106,2.928892,0.374000,0.626000,00:19


Better model found at epoch 0 with accuracy value: 0.07000000029802322.
Better model found at epoch 1 with accuracy value: 0.1459999978542328.
Better model found at epoch 2 with accuracy value: 0.2280000001192093.
Better model found at epoch 3 with accuracy value: 0.30399999022483826.
Better model found at epoch 4 with accuracy value: 0.36800000071525574.
Better model found at epoch 5 with accuracy value: 0.4259999990463257.
Better model found at epoch 6 with accuracy value: 0.4959999918937683.
Better model found at epoch 7 with accuracy value: 0.5540000200271606.
Better model found at epoch 8 with accuracy value: 0.6000000238418579.
Better model found at epoch 9 with accuracy value: 0.6259999871253967.
Better model found at epoch 10 with accuracy value: 0.6700000166893005.
Better model found at epoch 11 with accuracy value: 0.6859999895095825.
Better model found at epoch 12 with accuracy value: 0.7080000042915344.
Better model found at epoch 13 with accuracy value: 0.7139999866485596.

In [ ]:
nn.Sequential(*(list(learn.model.children())))

In [ ]:
nn.Sequential(*list(learn.model.children()))

In [ ]:
nn.Sequential(*list(learn.model.children()))[:-1]

In [ ]:
learn.summary()

In [ ]:
for param in learn.model.parameters():
    param.requires_grad = False